In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

In [2]:
# Import API key
from API_Keys import g_key

In [3]:
#load csv
city_df = pd.read_csv("../WeatherPy/city.csv")
city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Butaritari,3.0707,172.7902,300.80,77,71,2.46,KI,1618672740
1,1,Castro,-24.7911,-50.0119,293.25,76,97,1.33,BR,1618672741
2,2,Tiksi,71.6872,128.8694,258.98,96,100,4.93,RU,1618672742
3,3,Ginir,7.1333,40.7000,290.77,75,72,3.27,ET,1618672742
4,4,Fortuna,40.5982,-124.1573,280.15,100,90,2.06,US,1618672742


In [4]:
dropna_data = city_df.dropna()
del dropna_data["Unnamed: 0"]
dropna_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Butaritari,3.0707,172.7902,300.80,77,71,2.46,KI,1618672740
1,Castro,-24.7911,-50.0119,293.25,76,97,1.33,BR,1618672741
2,Tiksi,71.6872,128.8694,258.98,96,100,4.93,RU,1618672742
3,Ginir,7.1333,40.7000,290.77,75,72,3.27,ET,1618672742
4,Fortuna,40.5982,-124.1573,280.15,100,90,2.06,US,1618672742


In [5]:
gmaps.configure(g_key)
location = dropna_data[["Lat","Lng"]]
humidity = dropna_data["Humidity"]

In [6]:
figure_layout = {
    "width": "400px",
    "height": "300px",
    "border": "1px solid  black",
    "padding": "1px",
    "margin": "0 auto 0 auto"
}
fig = gmaps.figure(layout = figure_layout)
fig

Figure(layout=FigureLayout(border='1px solid  black', height='300px', margin='0 auto 0 auto', padding='1px', w…

In [7]:
heat = gmaps.heatmap_layer(location, weights=humidity, point_radius=1, dissipating=False, max_intensity=10)
fig.add_layer(heat)
fig

Figure(layout=FigureLayout(border='1px solid  black', height='300px', margin='0 auto 0 auto', padding='1px', w…

In [8]:
weather_criteria_df = dropna_data.loc[dropna_data["Max Temp"]<275]
temp_criteria = weather_criteria_df.loc[weather_criteria_df["Max Temp"]>200]
wind_criteria = temp_criteria.loc[temp_criteria["Wind Speed"]<10]
cloudiness_criteria = wind_criteria.loc[wind_criteria["Cloudiness"]==0]
cloudiness_criteria

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
411,Tahe,52.3333,124.7333,272.88,80,0,1.87,CN,1618672878


In [9]:
hotel_df = cloudiness_criteria[["City", "Lat", "Lng", "Max Temp", "Country"]].copy()
hotel_df["Hotel Name"] = " "
hotel_df

,City,Lat,Lng,Max Temp,Country,Hotel Name
411,Tahe,52.3333,124.7333,272.88,CN,


In [10]:
params = {
    "keyword": "Hotel",
    "types": "lodging",
    "radius": 5000,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    
    response = requests.get(base_url,params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except:
        print("Hotel not found")
hotel_df

,City,Lat,Lng,Max Temp,Country,Hotel Name
411,Tahe,52.3333,124.7333,272.88,CN,诚信商务酒店


In [11]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
hotel_markers = gmaps.marker_layer(locations,hotel_info)
fig.add_layer(hotel_markers)
fig

Figure(layout=FigureLayout(border='1px solid  black', height='300px', margin='0 auto 0 auto', padding='1px', w…